In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [64]:
df_train = pd.read_csv('/content/Corona_NLP_train.csv',encoding="latin-1")
df_test = pd.read_csv('/content/Corona_NLP_test.csv')

In [65]:
df_test = df_test.drop(['UserName','ScreenName','Location','TweetAt'],axis=1)
df_train = df_train.drop(['UserName','ScreenName','Location','TweetAt'],axis=1)

In [66]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [67]:
max_features = 10000
max_len = 200

In [68]:
# 1. Fit tokenizer on training data
tokenizer = Tokenizer(num_words=max_features, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train["OriginalTweet"])

# 2. Convert text to sequences of integers
x_train = tokenizer.texts_to_sequences(df_train["OriginalTweet"])
x_test = tokenizer.texts_to_sequences(df_test["OriginalTweet"])

# 3. Pad sequences to fixed length
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [69]:
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [70]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [71]:
from sklearn.preprocessing import LabelEncoder

num_classes = len(le.classes_)   # should be 5 in your case

# Labels (one-hot)
y_train = tf.keras.utils.to_categorical(le.fit_transform(df_train["Sentiment"]), num_classes)
y_test = tf.keras.utils.to_categorical(le.transform(df_test["Sentiment"]), num_classes)

# Model (update final layer)
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation="softmax"))   # <-- changed

model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])


In [72]:
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=3,
    validation_data=(x_test, y_test)
)


Epoch 1/3
 47/644 ━━━━━━━━━━━━━━━━━━━━ 6:33 660ms/step - accuracy: 0.2703 - loss: 1.5827

KeyboardInterrupt: 

In [ ]:
score, acc = model.evaluate(df_test["OriginalTweet"], df_test["Sentiment"], batch_size=64)
print("Test Loss:", score)
print("Test Accuracy:", acc)